<a href="https://colab.research.google.com/github/xN1ckuz/Crosswalks-Detection-using-YoloV5/blob/main/crosswalks_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

folder = 'project_VeP'#@param{type:"string"}

drive.mount('/content/gdrive')
print('Connecting to Drive....')

%cd /content/gdrive/MyDrive/$folder/
print('Done!')

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
import torch
print("Repo Clone Completed!")

In [ ]:
epoch = 250 #@param {type:"slider", min:100, max:1000, step:10}
training_option = 'Train new CNN' #@param ["Train new CNN", "Resume an interrupted Training"]
weight = 'yolov5m.pt' #@param["yolov5s.pt","yolov5m.pt","yolov5l.pt","yolov5x.pt"]
batch = 64 #@param {type:"slider", min:1, max:64, step:1}

if training_option == 'Train new CNN': 
  %rm -r /content/gdrive/MyDrive/$folder/training_results/exp
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --img 512 --batch $batch --epochs $epoch --data /content/gdrive/MyDrive/$folder/training_dataset/data.yaml --weights $weight --cache --project /content/gdrive/MyDrive/$folder/training_results/
else:
  !python /content/gdrive/MyDrive/$folder/yolov5/train.py --resume /content/gdrive/MyDrive/$folder/training_results/export/weights/last.pt 

In [ ]:
#@title This calculates the metrics on a series of images called 'test'
%rm -r /content/gdrive/MyDrive/$folder/test_results
!python /content/gdrive/MyDrive/$folder/yolov5/val.py --data /content/gdrive/MyDrive/$folder/test_dataset/data.yaml --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/test_results/

In [ ]:
#Printing result to confront
from PIL import Image

precision_training = '/content/gdrive/MyDrive/'+folder+'/training_results/exp/F1_curve.png'
precision_test = '/content/gdrive/MyDrive/'+folder+'/test_results/exp/F1_curve.png'

im1 = Image.open(precision_training)
im2 = Image.open(precision_test)

print("F1 Training")
display(im1)
print("F1 Test")
display(im2)

In [ ]:
video_name = 'video1.mp4' #@param {type:"string"}
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/videos/$video_name --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/detection_results/                        

In [ ]:
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/detection/test --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/detection_results/ --imgsz 512 --save-txt --save-conf --exist-ok

detect: weights=['/content/gdrive/MyDrive/project_VeP/training_results/exp/weights/best.pt'], source=/content/gdrive/MyDrive/project_VeP/detection/test, data=yolov5/data/coco128.yaml, imgsz=[512, 512], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/gdrive/MyDrive/project_VeP/detection_results/, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-244-g47233e1 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients
image 1/203 /content/gdrive/MyDrive/project_VeP/detection/test/Test (1).jpg: 288x512 2 crosswalks, Done. (0.022s)
image 2/203 /content/gdrive/MyDrive/project_VeP/detection/test/Test (10).jpg: 512x384 Done. (0.020s)
image 3/203 /content/gdrive/MyDrive/p

In [ ]:
#Da far vedere Test46

import glob
import os
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/labels/'
img_path = '/content/gdrive/MyDrive/'+folder+'/detection_results/exp/'

import os, glob
for filename in glob.glob(os.path.join(path, '*.txt')):
  with open(os.path.join(os.getcwd(),filename), 'r') as f:
    bounding_boxes = None
    image_name = os.path.basename(filename).replace('txt', 'jpg')
    for line in f:
      cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
      a = np.array([float(label_x),float(label_y),float(label_w),float(label_h)])
      bounding_boxes = (np.vstack((bounding_boxes, a)) if (bounding_boxes is not None) else a)
    
    #If there is only one bounding box, skip
    if(bounding_boxes.ndim == 1):
      continue
    
    #Image Elaboration
    image = cv.imread(img_path+image_name)
    [image_height, image_width, levels] = image.shape
    
    [x,y] = [image_height,image_width]
    [w,h] = [0,0]
    for i in range(0,bounding_boxes.shape[0]):
      center_x_1 = int(bounding_boxes[i,0]*image_width)
      center_y_1 = int(bounding_boxes[i,1]*image_height) 
      width_1 = int(bounding_boxes[i,2]*image_width) 
      height_1 = int(bounding_boxes[i,3]*image_height)
      vert_x = int(center_x_1 - (width_1/2))
      vert_y = int(center_y_1 - (height_1/2))
      vert_w = int(center_x_1 + (width_1/2))
      vert_h = int(center_y_1 + (height_1/2))
      if(vert_x < x):
        x = vert_x
      if(vert_y < y):
        y = vert_y
      if(vert_w > w):
        w = vert_w
      if(vert_h > h):
        h = vert_h
    
    #Create Rect
    cv.rectangle(image, (x,y), (w,h), (0,0,255), 8)
    
    #Saving Image
    print('Saving ' + image_name)
    cv.imwrite(img_path+image_name, image)

Saving Test (1).jpg
Saving Test (101).jpg
Saving Test (106).jpg
Saving Test (113).jpg
Saving Test (114).jpg
Saving Test (120).jpg
Saving Test (125).jpg
Saving Test (129).jpg
Saving Test (130).jpg
Saving Test (133).jpg
Saving Test (136).jpg
Saving Test (137).jpg
Saving Test (140).jpg
Saving Test (144).jpg
Saving Test (147).jpg
Saving Test (149).jpg
Saving Test (153).jpg
Saving Test (156).jpg
Saving Test (16).jpg
Saving Test (165).jpg
Saving Test (178).jpg
Saving Test (179).jpg
Saving Test (19).jpg
Saving Test (2).jpg
Saving Test (202).jpg
Saving Test (24).jpg
Saving Test (29).jpg
Saving Test (31).jpg
Saving Test (35).jpg
Saving Test (36).jpg
Saving Test (4).jpg
Saving Test (40).jpg
Saving Test (45).jpg
Saving Test (5).jpg
Saving Test (53).jpg
Saving Test (54).jpg
Saving Test (56).jpg
Saving Test (57).jpg
Saving Test (58).jpg
Saving Test (59).jpg
Saving Test (60).jpg
Saving Test (61).jpg
Saving Test (62).jpg
Saving Test (64).jpg
Saving Test (7).jpg
Saving Test (70).jpg
Saving Test (71).j